In [1]:
import pandas as pd
import re
import copy
from datetime import datetime

members = pd.read_csv('members.csv')
bookings = pd.read_csv('bookings.csv')
facilities = pd.read_csv('facilities.csv')

In [2]:
#simplejoin
sj = pd.merge(members, bookings, on='memid')
sj.loc[(sj['firstname'] == "David") & (sj['surname'] == "Farrell")].loc[:, ['starttime']]

,starttime
3915,2012-09-18 09:00:00
3916,2012-09-18 17:30:00
3917,2012-09-18 13:30:00
3918,2012-09-18 20:00:00
3919,2012-09-19 09:30:00
3920,2012-09-19 15:00:00
3921,2012-09-19 12:00:00
3922,2012-09-20 15:30:00
3923,2012-09-20 11:30:00
3924,2012-09-20 14:00:00


In [15]:
#simplejoin2
sj2 = pd.merge(facilities, bookings, on='facid')
sj2.loc[(pd.to_datetime(sj2['starttime']) >= datetime.strptime('2012-09-21 00:00:00', '%Y-%m-%d %H:%M:%S')) &
    (pd.to_datetime(sj2['starttime']) < datetime.strptime('2012-09-22 00:00:00', '%Y-%m-%d %H:%M:%S')) &
    sj2['name'].str.startswith('Tennis Court'), ['starttime', 'name']] \
.sort_values(by='starttime') \
.rename(columns={'starttime':'start'})

,start,name
350,2012-09-21 08:00:00,Tennis Court 1
737,2012-09-21 08:00:00,Tennis Court 2
351,2012-09-21 09:30:00,Tennis Court 1
738,2012-09-21 10:00:00,Tennis Court 2
739,2012-09-21 11:30:00,Tennis Court 2
352,2012-09-21 12:00:00,Tennis Court 1
353,2012-09-21 13:30:00,Tennis Court 1
740,2012-09-21 14:00:00,Tennis Court 2
354,2012-09-21 15:30:00,Tennis Court 1
741,2012-09-21 16:00:00,Tennis Court 2


In [21]:
#sub


In [5]:
#count3
members.loc[:, ['recommendedby','memid']] \
.groupby('recommendedby') \
.count() \
.rename(columns={'memid':'count'}) \
.sort_values(by='recommendedby')

,count
recommendedby,
1.0,5
2.0,3
3.0,1
4.0,2
5.0,1
6.0,1
9.0,2
11.0,1
13.0,2


In [6]:
#fachours
bookings.loc[:, ['facid', 'slots']] \
.groupby('facid') \
.sum() \
.rename(columns={'slots':'Total Slots'}) \
.sort_values(by='facid')

,Total Slots
facid,
0,1320
1,1278
2,1209
3,830
4,1404
5,228
6,1104
7,908
8,911


In [18]:
#fachoursbymonth
fachoursm = copy.copy(bookings)
fachoursm.loc[(pd.to_datetime(fachoursm['starttime']) >= datetime.strptime('2012-09-01 00:00:00', '%Y-%m-%d %H:%M:%S')) &
             (pd.to_datetime(fachoursm['starttime']) <= datetime.strptime('2012-09-30 23:59:59', '%Y-%m-%d %H:%M:%S')), 
             ['facid', 'slots']] \
.groupby('facid') \
.sum() \
.rename(columns={'slots':'Total Slots'}) \
.sort_values(by='Total Slots')

,Total Slots
facid,
5,122
3,422
7,426
8,471
6,540
2,570
1,588
0,591
4,648


In [48]:
#fachoursbymonth2
fachoursm2 = copy.copy(bookings)
fachoursm2['month'] = pd.to_datetime(fachoursm2['starttime']).dt.month
fachoursm2['year'] = pd.to_datetime(fachoursm2['starttime']).dt.year
fachoursm2.loc[(fachoursm2['year'] == 2012), ['facid', 'month', 'slots']] \
.groupby(['facid', 'month']) \
.sum() \
.rename(columns={'slots':'Total Slots'}) \
.sort_values(by=['facid', 'month'])

Total Slots
facid month             
0     7              270
      8              459
      9              591
1     7              207
      8              483
      9              588
2     7              180
      8              459
      9              570
3     7              104
      8              304
      9              422
4     7              264
      8              492
      9              648
5     7               24
      8               82
      9              122
6     7              164
      8              400
      9              540
7     7              156
      8              326
      9              426
8     7              117
      8              322
      9              471

In [56]:
#members1
bookings.loc[:, ['memid']].nunique().to_frame(name='count')

,count
memid,30


In [58]:
#nbooking
nb = pd.merge(members, bookings, on='memid')
nb.loc[(pd.to_datetime(nb['starttime']) >= datetime.strptime('2012-09-01 00:00:00', '%Y-%m-%d %H:%M:%S')), 
      ['surname', 'firstname', 'memid', 'starttime']] \
.groupby('memid') \
.min() \
.sort_values('memid')

,surname,firstname,starttime
memid,,,
0,GUEST,GUEST,2012-09-01 08:00:00
1,Smith,Darren,2012-09-01 09:00:00
2,Smith,Tracy,2012-09-01 11:30:00
3,Rownam,Tim,2012-09-01 16:00:00
4,Joplette,Janice,2012-09-01 15:00:00
5,Butters,Gerald,2012-09-02 12:30:00
6,Tracy,Burton,2012-09-01 15:00:00
7,Dare,Nancy,2012-09-01 12:30:00
8,Boothe,Tim,2012-09-01 08:30:00


In [59]:
#concat
c = copy.copy(members)
c['name'] = c['surname'] + ', ' + c['firstname']
c.loc[:, ['name']]

,name
0,"GUEST, GUEST"
1,"Smith, Darren"
2,"Smith, Tracy"
3,"Rownam, Tim"
4,"Joplette, Janice"
5,"Butters, Gerald"
6,"Tracy, Burton"
7,"Dare, Nancy"
8,"Boothe, Tim"
9,"Stibbons, Ponder"


In [69]:
#case
facilities.loc[(facilities['name'].str.match('^Tennis'))]

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200


In [75]:
#reg
members.loc[(members['telephone'].str.match('^\(.{3}\)')), ['memid', 'telephone']]

,memid,telephone
0,0,(000) 000-0000
3,3,(844) 693-0723
4,4,(833) 942-4710
5,5,(844) 078-4130
6,6,(822) 354-9973
7,7,(833) 776-4001
8,8,(811) 433-2547
9,9,(833) 160-3900
10,10,(855) 542-5251
11,11,(844) 536-8036


In [84]:
#substr
s = copy.copy(members)
s['letter'] = s['surname'].str.get(0)
s.loc[:, ['letter', 'surname']] \
.groupby('letter') \
.count() \
.rename(columns={'surname':'count'}) \
.sort_values(by='letter')

,count
letter,
B,5
C,2
D,1
F,2
G,2
H,1
J,3
M,1
O,1


In [24]:
members[(members['address'].str.contains('Boston', regex=False))]

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,NaN,2012-07-02 12:02:05
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,NaN,2012-07-03 09:32:15
5,5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1.0,2012-07-09 10:44:09
6,6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,NaN,2012-07-15 08:52:55
7,7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4.0,2012-07-25 08:59:12
12,12,Baker,Anne,"55 Powdery Street, Boston",80743,844-076-5141,9.0,2012-08-10 14:23:22
14,14,Smith,Jack,"252 Binkington Way, Boston",69302,(822) 163-3254,1.0,2012-08-10 16:22:05
17,17,Pinker,David,"5 Impreza Road, Boston",65332,811 409-6734,13.0,2012-08-16 11:32:47
18,20,Genting,Matthew,"4 Nunnington Place, Wingfield, Boston",52365,(811) 972-1377,5.0,2012-08-19 14:55:55
20,22,Coplin,Joan,"85 Bard Street, Bloomington, Boston",43533,(822) 499-2232,16.0,2012-08-29 08:32:41
